In [ ]:
import sqlite3

conn = sqlite3.connect('ex1.db')
c = conn.cursor()

c.execute(''' CREATE TABLE books (id integer , title text, price integer)
''')

c.execute(''' INSERT INTO books VALUES (1,'python 가이드',5000)
''')
c.execute(''' INSERT INTO books VALUES (2,'JS 가이드',7000)
''')

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('ex1.db')

# 커서 생성
cursor = conn.cursor()

# 데이터 조회
cursor.execute('SELECT * FROM books')
result = cursor.fetchall()

# 조회 결과 출력
for row in result:
    print(row)

# 연결 닫기
conn.close()

(1, 'python 가이드', 5000)
(2, 'JS 가이드', 7000)


In [ ]:
## 실습 2
import sqlite3

conn = sqlite3.connect('ex2.db')
c = conn.cursor()

c.execute('''
    CREATE TABLE books (id integer, title text, price integer)
''')

for i in range(10):
    c.execute(f"INSERT INTO books VALUES ({i}, 'python 가이드', {i*1000})")

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('ex2.db')
c = conn.cursor()

# 데이터 조회
for row in c.execute('SELECT * FROM books ORDER BY id'):
    print(row)

# 연결 종료
conn.close()

(0, 'python 가이드', 0)
(1, 'python 가이드', 1000)
(2, 'python 가이드', 2000)
(3, 'python 가이드', 3000)
(4, 'python 가이드', 4000)
(5, 'python 가이드', 5000)
(6, 'python 가이드', 6000)
(7, 'python 가이드', 7000)
(8, 'python 가이드', 8000)
(9, 'python 가이드', 9000)


In [ ]:
## 실습 3
import sqlite3

data = {
    'id': [1, 2, 3],
    'title': ['1', '2', '3'],
    'content': ['11', '22', '33', '44', '55']
}

# 데이터베이스에 연결
conn = sqlite3.connect('ex3.db')

# 커서 생성
cursor = conn.cursor()

# post 테이블 생성
cursor.execute('CREATE TABLE post (id INTEGER, title TEXT, content TEXT)')

# 데이터 삽입
for i in range(len(data['id'])):
    cursor.execute('INSERT INTO post VALUES (?, ?, ?)', (data['id'][i], data['title'][i], data['content'][i]))

# 커밋(변경 사항 저장)
conn.commit()

# 연결 닫기
conn.close()

In [ ]:
conn = sqlite3.connect('ex3.db')
c = conn.cursor()

# 데이터 조회
for row in c.execute('SELECT * FROM post'):
    print(row)

# 연결 종료
conn.close()

(1, '1', '11')
(2, '2', '22')
(3, '3', '33')


In [1]:
'''요구사항1. book 크롤링한 데이터를 데이터베이스에 담아야 합니다.
        - 데이터베이스를 생성
        - 크롤링
        - id, title, price, author
    요구사항2. 데이터베이스를 읽습니다.
    요구사항3. 데이터베이스를 JSON 형태로 출력해야 합니다. 출력 이름은 output.json
'''


import sqlite3
import requests
from bs4 import BeautifulSoup

url = 'https://paullab.co.kr/bookservice/'
response = requests.get(url)
response.encoding = 'utf-8'
html = response.text

soup = BeautifulSoup(html, 'html.parser')

title_soup = soup.select('.book_name')
price_soup = soup.select('.book_info:nth-of-type(1)')
author_soup = soup.select('.book_info:nth-of-type(2)')

title_data = [data.text  for data in title_soup]
price_data = []
author_data = [data.text[4:]  for data in author_soup]

for data in price_soup:
    data = data.text[4:-1].replace(',','')
    if not data.isnumeric():
        data = 0
    else:
        data = int(data)
    price_data.append(data)

conn = sqlite3.connect('book.sqlite3')
c = conn.cursor()

c.execute('''
    CREATE TABLE books (id integer PRIMARY KEY AUTOINCREMENT, title text, price integer, author text )
''')

for i in range(len(title_data)):
    c.execute(f'INSERT INTO books(title, price, author) VALUES("{title_data[i]}", {price_data[i]}, "{author_data[i]}")')
conn.commit()

fetch_data = c.execute('SELECT * FROM books')
table_name = c.description
fetch_data = fetch_data.fetchall()
conn.close()

import json
FILE_NAME = "./output.json"

data_list = []
for i in range(len(title_data)):
    data = {}
    for j in range(len(table_name)):
        data[table_name[j][0]] = fetch_data[i][j]
    data_list.append(data)

f = open(FILE_NAME, 'w', encoding='utf-8')
f.write(json.dumps(data_list, ensure_ascii=False))
f.close()

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import json

conn = sqlite3.connect('books2.db')
c = conn.cursor()

c.execute('CREATE TABLE books (id integer, title text, price integer, author text)')

paullab_url = 'http://paullab.co.kr/bookservice/'
response = requests.get(paullab_url)
response.encoding = 'utf-8'
html = response.text

soup = BeautifulSoup(html, 'html.parser')
data = []

book_name_list = soup.select('.book_detail > .book_name')
book_price_list = soup.select('.book_detail > .book_info')[::3]
book_author_list = soup.select('.book_detail > .book_info')[1::3]

for i in range(len(book_name_list)):
    name = book_name_list[i].text
    price = book_price_list[i].text[4:].replace('원', '').replace(',', '').replace('무료', '0')
    author = book_author_list[i].text[4:]
    c.execute('INSERT INTO books VALUES (?, ?, ?, ?)', (i, name, price, author))

conn.commit()

books = []

for row in c.execute('SELECT * FROM books'):
    books.append({
        'id': row[0],
        'title': row[1],
        'price': row[2],
        'author': row[3],
    })

with open('.\output.json', 'w', encoding='utf-8') as file:
    json.dump(books, file, indent="\t", ensure_ascii=False)

conn.close()